In [1]:
# 라이브러리 가져오기
import re
import json
import numpy as np
import pandas as pd

# TF-IDF 전처리

In [2]:
# 데이터 가져오기
df = pd.read_json("yes24.latest.json")

ValueError: Expected object or value

## 프리뷰 전처리

In [ ]:
# 한국어와 공백이 아닌 모든 문자를 공백으로 바꾼다.
df.preview = df.preview.str.replace(r"[^가-힣 \s]", " ", regex=True)

# 여러개의 공백을 하나의 공백으로 바꾼다
df.preview = df.preview.str.replace(r"\s+", " ", regex=True)

# 문장 앞 뒤의 공백을 제거한다
df.preview = df.preview.str.strip()

# 빈 공백을 nan 으로 바꾼다
df.preview = df.preview.replace("", np.nan)

# 결측치 제거
df = df.dropna()

## 타이틀 전처리

In [ ]:
# 대괄호 내부에 문자를 제거한다
df.title = df.title.str.replace(r"\[.+\]", "", regex=True)

# 여러개의 공백을 하나의 공백으로 바꾼다
df.title = df.title.str.replace(r"\s+", " ", regex=True)

# 문장 앞 뒤의 공백을 제거한다
df.title = df.title.str.strip()

# 제목 중복을 제거한다
df = df.drop_duplicates(["title"])

# 미리보기 중복을 제거한다
df = df.drop_duplicates(["preview"])

In [ ]:
# 1000자 미만 삭제
df = df[df.preview.str.len() > 1000]

In [ ]:
# 인덱스를 재설정한다.
df = df.reset_index(drop=True, inplace=False)

In [ ]:
df

# 명사 형태소 분석

In [ ]:
# 라이브러리 가져오기
from tqdm.auto import tqdm
from konlpy.tag import Mecab

# 미캡 객체 생성
tagger = Mecab(dicpath=r"C:\mecab\mecab-ko-dic")

In [ ]:
# 분석할 데이터 담을 변수
analyzed_data = []

# 미리보기 데이터 조회
for preview in tqdm(df.preview):
    
    # 명사만 추출
    tag_result = tagger.nouns(preview)
    
    # 각 단어를 공백을 사이에 두고 하나로 합침
    result = " ".join(tag_result)

    # 데이터를 변수에 담음
    analyzed_data.append(result)

# TF-IDF유사도 분석

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
# TF-IDF를 벡터로 분석
tfidf_matrix = TfidfVectorizer().fit_transform(analyzed_data)
print('TF-IDF 행렬의 크기(shape) :', tfidf_matrix.shape)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
# TF-IDF 벡터의 cosine 유사도 계산
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
print('코사인 유사도 연산 결과 :' ,cosine_sim.shape)

In [ ]:
# 제목을 TF-IDF 안의 인덱스로 변환하기 위한 준비
title_to_index = dict(zip(df['title'], df.index))

In [ ]:
def get_recommendations(title, cosine_sim=cosine_sim):
    # 선택한 소설의 타이틀로부터 해당 소설의 인덱스를 받아온다.
    idx = title_to_index[title]

    # 해당 소설과 모든 소설과의 유사도를 가져온다.
    sim_scores = list(enumerate(cosine_sim[idx]))

    # 유사도에 따라 소설들을 정렬한다.
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # 가장 유사한 10개의 소설을 받아온다.
    sim_scores = sim_scores[1:11]

    # 가장 유사한 10개의 소설의 인덱스를 얻는다.
    novel_indices = [idx[0] for idx in sim_scores]

    # 가장 유사한 10개의 소설의 제목을 리턴한다.
    # 인덱스 리스트에서 제목 리스트로 변환한다.
    title_list = df['title'].iloc[novel_indices]
    # 소숫점 유사도를 퍼센트로 변환한다.
    score_list = [round(score[1], 3) * 100 for score in sim_scores]
    # 보기 편하도록 데이터 프레임으로 바꿔 반환한다.
    return pd.DataFrame({"제목": title_list, "score": score_list})

# TF-IDF 예시

In [ ]:
# 제목으로 검색
get_recommendations('왕을 찾아서')

# 형태소 전체 분석

In [ ]:
# 라이브러리 가져오기
from tqdm.auto import tqdm
from konlpy.tag import Mecab
from gensim.models.doc2vec import TaggedDocument


tagger = Mecab(dicpath=r"C:\mecab\mecab-ko-dic")

In [ ]:
tagged_corpus_list = []

# 한 행씩 순환
for index, row in tqdm(df.iterrows(), total=len(df)):
    preview = row['preview']
    title = row['title']
    
    # 문장을 형태소로 자름
    words = tagger.morphs(preview)
    
    # 각 행을 문서 객체로 변환
    td = TaggedDocument(tags=[title], words=words)
    
    # 데이터를 변수에 담음
    tagged_corpus_list.append(td)

print('문서의 수 :', len(tagged_corpus_list))

# Doc2Vec 모델 학습 + 저장

In [ ]:
from gensim.models import doc2vec

In [ ]:
model = doc2vec.Doc2Vec(vector_size=300, alpha=0.025, min_alpha=0.025, workers=4, window=8)

# Vocabulary 빌드(데이터 주입)
model.build_vocab(tagged_corpus_list)
print(f"Tag Size: {len(model.docvecs.doctags.keys())}", end=' / ')

# Doc2Vec 학습
model.train(tagged_corpus_list, total_examples=model.corpus_count, epochs=50)

# 모델 저장
model.save('dart.doc2vec')

# 저장한 모델을 불러옴
gensim 버전이 다르면 불러오지 못함

In [ ]:
model = doc2vec.Doc2Vec.load("dart.doc2vec")

# Doc2Vec 예시

In [ ]:
similar_doc = model.docvecs.most_similar('왕을 찾아서')
pd.DataFrame(similar_doc, columns=["제목", "유사도"])